<a href="https://colab.research.google.com/github/bcb12/agroanalysis/blob/main/Agroanalysis_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Anotaciones para futuras fases:

*   Identificar elementos nulos
*   Sustituir símbolos problemáticos ("," en lugar de "." como separador de decimales)
*   Sustituir separador de elementos "|" por ";"
*   Eliminar los símbolos ;; del final de cada línea en el Dataset 1



Import de librerías externas

In [ ]:
# Imports
import pandas as pd
import io
import os
import missingno as msno
import datetime as dt
from google.colab import drive

(LEGACY) Cargar archivo desde el equipo.

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('"{nombre}" {longitud}'.format(nombre=fn, longitud=len(uploaded[fn])))

Saving Dataset1_Datos_Consumo_Alimentario_MAPA_por_CCAA_CONVERTED.txt to Dataset1_Datos_Consumo_Alimentario_MAPA_por_CCAA_CONVERTED.txt
"Dataset1_Datos_Consumo_Alimentario_MAPA_por_CCAA_CONVERTED.txt" 1829234


Montar directorio de Drive y cargar los datasets.

In [ ]:
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/Datasets/Converted Datasets')
print(os.getcwd())

dataset_1 = pd.read_csv("Dataset1_CONVERTED.txt")
dataset_3a = pd.read_csv("Dataset3a_CONVERTED.txt", encoding='utf8')
dataset_3b = pd.read_csv("Dataset3b_CONVERTED.txt")
dataset_4 = pd.read_csv("Dataset4_CONVERTED.txt")
dataset_5 = pd.read_csv("Dataset5_CONVERTED.txt")
#dataset_5 = pd.read_csv("/content/drive/MyDrive/Datasets/D5 Processing/D5.csv")

df_1 = pd.DataFrame(dataset_1)
df_3a = pd.DataFrame(dataset_3a)
df_3b = pd.DataFrame(dataset_3b)
df_4 = pd.DataFrame(dataset_4)
df_5 = pd.DataFrame(dataset_5)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Datasets/Converted Datasets


### Diccionario de nombres de productos

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
source_products = pd.read_csv("Products Dictionary/FV_Cambios_fix.csv")
source_products_df = pd.DataFrame(source_products)

# Dataset 1

Corregimos los nombres de los productos para que se llamen como los de los demás datasets.
Utilizamos como diccionario la fila de origen "*Dataset 1*" y reemplazamos sus valores por "*Dataset 1 N*" del documento **FV_Cambios.csv**.

In [ ]:
source_dict_ds1 = source_products_df[['Dataset 1', 'Dataset 1 N']]
dict_1 = dict(source_dict_ds1.values)
df_1 = df_1.replace({"Producto":dict_1})

Eliminamos las filas que contienen nulos en la columna **Producto**.

In [ ]:
df_1.dropna(axis=0, subset=['Producto'], inplace=True)

Nos quedamos únicamente con los valores que corresponden al total a nivel nacional en España, descartando los relativos a las comunidades autónomas.

In [ ]:
df_1 = df_1[df_1['CCAA'] == 'Total Nacional']

Cambiamos el formato de los meses, de su nombre en texto a la numeración correspondiente, y después pasamos las columnas de año y mes a tipo entero (int).

In [ ]:
months_dict  = {'Enero':1, 'Febrero':2, 'Marzo':3, 'Abril':4, 'Mayo':5, 'Junio':6, 'Julio':7, 'Agosto':8, 'Septiembre':9, 'Octubre':10, 'Noviembre':11, 'Diciembre':12}
df_1.Mes = df_1.Mes.map(months_dict)
df_1['Mes']=df_1['Mes'].astype(int)
df_1['Año']=df_1['Año'].astype(int)

Eliminamos las columnas *CCAA*, *Valor (miles de €)* y* Gasto per capita*.

In [ ]:
df_1.drop(['CCAA','Valor (miles de €)', 'Gasto per capita'], axis=1, inplace=True) # inplace=True

Renombramos las columnas *Año*, *Mes*, *Producto*, *Volumen (miles de kg)*, *Precio medio kg*, *Penetracion (%)* y *Consumo per capita* por *Year*, *Month*, *Product*, *Volume kg*, *Avg Weight*. *Penetration (%)* y *Per Capita Consum* respectivamente.

In [ ]:
df_1.rename(columns = {'Año': 'Year', 'Mes': 'Month', 'Producto': 'Product', 'Volumen (miles de kg)': 'Volume kg', 'Precio medio kg': 'Avg Weight', 'Penetracion (%)': 'Penetration (%)', 'Consumo per capita': 'Per Capita Consum'}, inplace=True)
df_1.set_index(['Product', 'Year', 'Month'])
df_1 = df_1.groupby(['Product', 'Year', 'Month']).agg({'Volume kg':'sum','Avg Weight':'mean', 'Penetración (%)': 'mean', 'Per Capita Consum': 'mean'}).reset_index()

Resultado

In [ ]:
df_1

,Product,Year,Month,Volume kg,Avg Weight,Penetración (%),Per Capita Consum
0,AGUACATE,2018,1,4378.92,3.35,23.49,0.10
1,AGUACATE,2018,2,4086.17,3.32,23.05,0.09
2,AGUACATE,2018,3,4471.08,3.59,24.01,0.10
3,AGUACATE,2018,4,4705.31,3.48,24.43,0.10
4,AGUACATE,2018,5,5185.62,3.30,26.15,0.11
...,...,...,...,...,...,...,...
885,ZANAHORIA,2020,2,13852.21,0.96,55.74,0.31
886,ZANAHORIA,2020,3,17026.27,0.97,62.82,0.38
887,ZANAHORIA,2020,4,20621.82,0.99,68.42,0.47
888,ZANAHORIA,2020,5,18777.18,0.96,64.73,0.42


# Dataset 3a

Corregimos los nombres de los productos para que se llamen como los de los demás datasets.
Utilizamos como diccionario la fila de origen "*Dataset 3a*" y reemplazamos sus valores por "*Dataset 3a N*" del documento **FV_Cambios.csv**.

In [ ]:
source_dict_ds3a = source_products_df[['Dataset 3a', 'Dataset 3a N']]
dict_3a = dict(source_dict_ds3a.values)
df_3a = df_3a.replace({"product":dict_3a})

Eliminamos las filas que contienen nulos en la columna **product**.

In [ ]:
df_3a.dropna(axis=0, subset=['product'], inplace=True)

Eliminamos las columnas *Variedad*, *Origen*, *Unidad*, *Familia*, *Price min* y *Price max*, y sumamos los valores de las filas agrupando por producto y fecha (mes y año).

In [ ]:
df_3a.drop(['variedad','origen', 'Unidad', 'familia','price_min','price_max'], axis=1, inplace=True) # inplace=True
df_3a.rename(columns = {'YEAR': 'Year', 'MONTH': 'Month', 'product': 'Product'}, inplace=True)
df_3a = df_3a.groupby(['Product', 'Year', 'Month']).agg({'price_mean': 'mean', 'Volumen': 'sum'}).reset_index()

Resultado

In [ ]:
df_3a

,Product,Year,Month,price_mean,Volumen
0,AGUACATE,2018,1,4.000000,1245632
1,AGUACATE,2018,2,4.112269,1375868
2,AGUACATE,2018,3,4.439931,1271984
3,AGUACATE,2018,4,4.447526,1474290
4,AGUACATE,2018,5,4.163207,1714863
...,...,...,...,...,...
853,ZANAHORIA,2020,7,0.675000,5197152
854,ZANAHORIA,2020,8,0.675000,4881272
855,ZANAHORIA,2020,9,0.675000,5999779
856,ZANAHORIA,2020,10,0.675000,6343144


# Dataset 3b

Corregimos los nombres de los productos para que se llamen como los de los demás datasets.
Utilizamos como diccionario la fila de origen "*Dataset 3b*" y reemplazamos sus valores por "*Dataset 3b N*" del documento **FV_Cambios.csv**.

In [ ]:
source_dict_ds3b = source_products_df[['Dataset 3b', 'Dataset 3b N']]
dict_3b = dict(source_dict_ds3b.values)
df_3b = df_3b.replace({"product":dict_3b})

Eliminamos las filas que contienen nulos en la columna **product**.

In [ ]:
df_3b.dropna(axis=0, subset=['product'], inplace=True)

Eliminamos las columnas *Origen*, *Unidad* y *Familia*, y sumamos los valores de las filas agrupando por producto y fecha (mes y año).

In [ ]:
df_3b.drop(['origen','Unidad', 'familia'], axis=1, inplace=True) # inplace=True
df_3b.rename(columns = {'YEAR': 'Year', 'MONTH': 'Month', 'product': 'Product'}, inplace=True)
df_3b = df_3b.groupby(['Product', 'Year', 'Month']).agg({'price_mean': 'mean', 'Volumen': 'sum'}).reset_index()

Resultado

In [ ]:
df_3b

,Product,Year,Month,price_mean,Volumen
0,AGUACATE,2018,1,3.230595,217134
1,AGUACATE,2018,2,3.341602,251629
2,AGUACATE,2018,3,3.372336,308587
3,AGUACATE,2018,4,3.114612,405048
4,AGUACATE,2018,5,3.019948,670959
...,...,...,...,...,...
869,ZANAHORIA,2020,7,0.500000,1540542
870,ZANAHORIA,2020,8,0.500000,1060692
871,ZANAHORIA,2020,9,0.500000,1807343
872,ZANAHORIA,2020,10,0.561071,1875883


# Dataset 4

Renombramos los nombres de los países que no coinciden con los del dataset 5, y eliminamos las filas que pertenecen a "*European Union*".

In [ ]:
# Fix Belgium name
df_4['REPORTER'].replace(r'Belgium.*', 'Belgium', regex = True, inplace = True)

# Fix France (France is unfixable)
df_4['REPORTER'].replace(r'France.*', 'France', regex = True, inplace = True)

# Fix Germany
df_4['REPORTER'].replace(r'Germany.*', 'Germany', regex = True, inplace = True)

# Fix Ireland
df_4['REPORTER'].replace(r'Ireland.*', 'Ireland', regex = True, inplace = True)

# Fix Italy
df_4['REPORTER'].replace(r'Italy.*', 'Italy', regex = True, inplace = True)

# Fix Spain
df_4['REPORTER'].replace(r'Spain.*', 'Spain', regex = True, inplace = True)

# Drop rows where REPORTER is "European Union"
df_4 = df_4[df_4["REPORTER"].str.contains("European Union.*") == False]


Eliminamos registros referentes a años completos y cambiamos el formato de la columna PERIOD de String a Datetime


In [ ]:
# Rows that represent yearly values
regex = '[asd|asd|asd]'
format = '[A-Z][a-z]{2}\.-[A-Z][a-z]{2}\. 20[0-9]{2}'
df_4 = df_4[df_4['PERIOD'].str.contains(format,regex=True) == False]
df_4['PERIOD'] = pd.to_datetime(df_4['PERIOD'], format = '%b. %Y')

<ipython-input-113-43cdccd214f8>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_4['PERIOD'] = pd.to_datetime(df_4['PERIOD'], format = '%b. %Y')


Transformar la columna PERIOD a las columnas Mes y Año, como en los demás datasets.

In [ ]:
df_4.insert(0, 'Year', df_4['PERIOD'].dt.year)
df_4.insert(1, 'Month', df_4['PERIOD'].dt.month)
del df_4['PERIOD']

Corregimos los nombres de los productos para que se llamen como los de los demás datasets.
Utilizamos como diccionario la fila de origen "*Dataset 4*" y reemplazamos sus valores por "*Dataset 4 N*" del documento **FV_Cambios.csv**.

In [ ]:
source_dict_ds4 = source_products_df[['Dataset 4', 'Dataset 4 N']]
dict_4 = dict(source_dict_ds4.values)
df_4 = df_4.replace({"PRODUCT":dict_4})

Eliminamos las filas que contienen nulos en la columna **PRODUCT**.

In [ ]:
df_4.dropna(axis=0, subset=['PRODUCT'], inplace=True)

Descartamos las filas cuyo indicador es **VALUE_IN_EUROS** o cuyo valor para flow es **EXPORT**.

In [ ]:
df_4 = df_4[df_4.INDICATORS.isin(['VALUE_IN_EUROS']) == False]
df_4 = df_4[df_4.FLOW.isin(['EXPORT']) == False]

Reemplazamos los valores nulos expresados con el valor "**:**" por **0**.

In [ ]:
df_4['Value'].replace(r':', 0, regex = True, inplace = True)

Eliminamos las columnas *PARTNER*, *FLOW* e *INDICATORS*.

In [ ]:
df_4.drop(['PARTNER','FLOW', 'INDICATORS'], axis=1, inplace=True)

Renombramos la columna *REPORTER* por *P_origen*

In [ ]:
df_4.rename(columns = {'PRODUCT': 'Product', 'REPORTER': 'P_origen'}, inplace=True)

Resultado

In [ ]:
df_4

,Year,Month,P_origen,Product,Value
1,2018,1,Austria,PLATANO,0
5,2018,2,Austria,PLATANO,0
9,2018,3,Austria,PLATANO,0
13,2018,4,Austria,PLATANO,0
17,2018,5,Austria,PLATANO,4
...,...,...,...,...,...
360809,2020,7,Sweden,TOMATE,4547
360813,2020,8,Sweden,TOMATE,7039
360817,2020,9,Sweden,TOMATE,4767
360821,2020,10,Sweden,TOMATE,7824


# Dataset 5

Se tienen nulos en los primeros dias del dataset dentro de la columna de casos acumulativos, esto se debe a que está calculado con los datos de 15 días antes, como no hay 15 días anteriores, es imposible calcularlos.

Creemos que esos datos no son estrictamente necesarios y se podrían eliminar

In [ ]:
df_5.drop(df_5[((df_5.day < 14) & (df_5.month == 1))].index, inplace=True,axis = 0)

Convertimos las fechas a formato datetime

In [ ]:
df_5['dateRep'] = pd.to_datetime(df_5['dateRep'])

Eliminamos las columnas *Cumulative_number_for_14_days_of_COVID-19_cases_per_100000*, *dateRep* y *popData2019*, *countryterritoryCode*, *geoId* y *continentExp*

In [ ]:
df_5 = df_5.drop(columns = ["day","Cumulative_number_for_14_days_of_COVID-19_cases_per_100000","dateRep","popData2019","countryterritoryCode","geoId","continentExp"])

Descartamos las filas en las que el país/territorio es *Cases_on_an_international_conveyance_Japan*

In [ ]:
df_5 = df_5[df_5.countriesAndTerritories != "Cases_on_an_international_conveyance_Japan"]

Renombramos las columnas *year*, *month* y *countriesAndTerritories* a *YEAR*, *MONTH* y *P_origen*, respectivamente

In [ ]:
df_5.rename(columns = {'year': 'Year', 'month': 'Month', 'countriesAndTerritories': 'P_origen', 'cases': 'Cases', 'deaths': 'Deaths'}, inplace=True)

Agrupamos por *año*, *mes* y *país*, y agregamos en las columnas **cases** y **deaths**

In [ ]:
df_5 = df_5.groupby(["Month","Year","P_origen"]).agg({'Cases':'sum','Deaths':'sum'}).reset_index()

Añadimos una columna con los datos de casos y muertes sumados

In [ ]:
df_5['Cases + Deaths'] = df_5["Cases"] + df_5["Deaths"]

In [ ]:
countries = ['Afghanistan', 'Algeria', 'Armenia', 'Australia',
       'Azerbaijan', 'Bahrain', 'Belarus','Brazil',
       'Cambodia', 'Canada', 'China',
       'Dominican_Republic', 'Ecuador', 'Egypt',
       'Georgia', 'Iceland', 'India',
       'Indonesia', 'Iran', 'Iraq', 'Israel', 'Japan',
       'Kuwait', 'Lebanon', 'Malaysia',
       'Mexico', 'Monaco', 'Nepal', 'New_Zealand',
       'Nigeria', 'North_Macedonia', 'Norway', 'Oman', 'Pakistan',
       'Philippines', 'Qatar', 'Russia', 'San_Marino',
       'Singapore', 'South_Korea', 'Sri_Lanka',
       'Switzerland', 'Taiwan', 'Thailand', 'United_Arab_Emirates',
       'United_Kingdom', 'United_States_of_America', 'Vietnam', 'Albania',
       'Andorra', 'Angola', 'Anguilla', 'Antigua_and_Barbuda',
       'Argentina', 'Aruba', 'Bahamas', 'Bangladesh', 'Barbados',
       'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bolivia',
       'Bosnia_and_Herzegovina', 'British_Virgin_Islands',
       'Brunei_Darussalam', 'Burkina_Faso', 'Cameroon',
       'Cape_Verde', 'Cayman_Islands', 'Central_African_Republic', 'Chad',
       'Chile', 'Colombia', 'Congo', 'Costa_Rica', 'Cote_dIvoire', 'Cuba',
       'CuraÃ§ao', 'Democratic_Republic_of_the_Congo',
       'Djibouti', 'Dominica', 'El_Salvador', 'Equatorial_Guinea',
       'Eritrea', 'Eswatini', 'Ethiopia', 'Faroe_Islands', 'Fiji',
       'French_Polynesia', 'Gabon', 'Gambia', 'Ghana', 'Gibraltar',
       'Greenland', 'Grenada', 'Guam', 'Guatemala', 'Guernsey', 'Guinea',
       'Guinea_Bissau', 'Guyana', 'Haiti', 'Holy_See', 'Honduras',
       'Isle_of_Man', 'Jamaica', 'Jersey', 'Jordan',
       'Kazakhstan', 'Kenya', 'Kosovo', 'Kyrgyzstan', 'Laos',
       'Liberia', 'Libya', 'Liechtenstein', 'Madagascar', 'Maldives',
       'Mali', 'Mauritania', 'Mauritius', 'Moldova', 'Mongolia',
       'Montenegro', 'Montserrat', 'Morocco', 'Mozambique', 'Myanmar',
       'Namibia', 'New_Caledonia', 'Nicaragua', 'Niger',
       'Northern_Mariana_Islands', 'Palestine', 'Panama',
       'Papua_New_Guinea', 'Paraguay', 'Peru',
       'Puerto_Rico', 'Rwanda', 'Saint_Kitts_and_Nevis', 'Saint_Lucia',
       'Saint_Vincent_and_the_Grenadines', 'Saudi_Arabia', 'Senegal',
       'Serbia', 'Seychelles', 'Sint_Maarten',
       'Somalia', 'South_Africa', 'Sudan', 'Suriname', 'Syria',
       'Timor_Leste', 'Togo', 'Trinidad_and_Tobago', 'Tunisia', 'Turkey',
       'Turks_and_Caicos_islands', 'Uganda', 'Ukraine',
       'United_Republic_of_Tanzania', 'United_States_Virgin_Islands',
       'Uruguay', 'Uzbekistan', 'Venezuela', 'Zambia', 'Zimbabwe',
       'Bonaire. Saint Eustatius and Saba', 'Botswana', 'Burundi',
       'Falkland_Islands_(Malvinas)', 'Malawi', 'Sao_Tome_and_Principe',
       'Sierra_Leone', 'South_Sudan', 'Western_Sahara', 'Yemen',
       'Comoros', 'Lesotho', 'Tajikistan', 'Marshall_Islands',
       'Solomon_Islands', 'Wallis_and_Futuna', 'Vanuatu']

mask = df_5['P_origen'].isin(countries)

df_5.drop(df_5[df_5['P_origen'].isin(countries)].index, inplace = True)

Resultado

In [ ]:
df_5['P_origen'].unique()

array(['Austria', 'Belgium', 'Croatia', 'Czechia', 'Denmark', 'Estonia',
       'Finland', 'France', 'Germany', 'Greece', 'Ireland', 'Italy',
       'Lithuania', 'Luxembourg', 'Netherlands', 'Romania', 'Spain',
       'Sweden', 'Bulgaria', 'Cyprus', 'Hungary', 'Latvia', 'Malta',
       'Poland', 'Portugal', 'Slovakia', 'Slovenia'], dtype=object)

# Join/Merge

Merge de todos los datasets. Los dataset *1*, *3a*, *3b* y *4* por las columnas **Product**, **Year** y **Month**, y el *5* por **Year**, **Month** y **P_Origen**

In [ ]:
df = pd.merge(df_3a, df_3b, on=['Product', 'Year', 'Month'], how='outer')

df = pd.merge(df, df_1, on=['Product', 'Year', 'Month'], how='outer')

df = pd.merge(df, df_4, on=['Product','Year','Month'], how='outer')

df = pd.merge(df, df_5, on=['Year','Month','P_origen'], how='left')

Juntar columnas price_mean en una sola haciendo la media entre ellas

In [ ]:
df['price_mean_x'] = (df['price_mean_x']+df['price_mean_y'])/2
df.rename(columns = {'price_mean_x': 'Price Mean'}, inplace=True)
df.drop(columns = ['Volumen_x','Volumen_y', 'price_mean_y'], inplace=True)

Resultado

In [ ]:
columns = ['Price Mean', 'Volume kg', 'Avg Weight', 'Penetración (%)', 'Per Capita Consum']
df.dropna(subset=columns, how='all')

,Product,Year,Month,Price Mean,Volume kg,Avg Weight,Penetración (%),Per Capita Consum,P_origen,Value,Cases,Deaths,Cases + Deaths
0,AGUACATE,2018,1,3.615298,4378.92,3.35,23.49,0.10,NaN,NaN,NaN,NaN,NaN
1,AGUACATE,2018,2,3.726935,4086.17,3.32,23.05,0.09,NaN,NaN,NaN,NaN,NaN
2,AGUACATE,2018,3,3.906133,4471.08,3.59,24.01,0.10,NaN,NaN,NaN,NaN,NaN
3,AGUACATE,2018,4,3.781069,4705.31,3.48,24.43,0.10,NaN,NaN,NaN,NaN,NaN
4,AGUACATE,2018,5,3.591577,5185.62,3.30,26.15,0.11,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
25768,PERA,2020,7,NaN,10791.59,1.93,NaN,0.24,NaN,NaN,NaN,NaN,NaN
25769,PERA,2020,8,NaN,13387.91,1.83,NaN,0.30,NaN,NaN,NaN,NaN,NaN
25770,PERA,2020,9,NaN,20333.21,1.71,NaN,0.46,NaN,NaN,NaN,NaN,NaN
25771,PERA,2020,10,NaN,26084.68,1.70,NaN,0.59,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.dropna(axis=0, subset=['P_origen'], inplace=True)

In [ ]:
df = df.fillna(0)
df

,Product,Year,Month,Price Mean,Volume kg,Avg Weight,Penetración (%),Per Capita Consum,P_origen,Value,Cases,Deaths,Cases + Deaths
35,AJO,2018,1,1.75271,2572.64,4.17,34.75,0.06,Austria,651,0.0,0.0,0.0
36,AJO,2018,1,1.75271,2572.64,4.17,34.75,0.06,Belgium,2215,0.0,0.0,0.0
37,AJO,2018,1,1.75271,2572.64,4.17,34.75,0.06,Bulgaria,230,0.0,0.0,0.0
38,AJO,2018,1,1.75271,2572.64,4.17,34.75,0.06,Croatia,171,0.0,0.0,0.0
39,AJO,2018,1,1.75271,2572.64,4.17,34.75,0.06,Cyprus,0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
25822,PATATAS,2020,12,0.00000,0.00,0.00,0.00,0.00,Slovenia,0,0.0,0.0,0.0
25823,PATATAS,2020,12,0.00000,0.00,0.00,0.00,0.00,Spain,0,0.0,0.0,0.0
25824,PATATAS,2020,12,0.00000,0.00,0.00,0.00,0.00,Spain,0,0.0,0.0,0.0
25825,PATATAS,2020,12,0.00000,0.00,0.00,0.00,0.00,Sweden,0,0.0,0.0,0.0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25569 entries, 35 to 25826
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Product            25569 non-null  object 
 1   Year               25569 non-null  int64  
 2   Month              25569 non-null  int64  
 3   Price Mean         25569 non-null  float64
 4   Volume kg          25569 non-null  float64
 5   Avg Weight         25569 non-null  float64
 6   Penetración (%)    25569 non-null  float64
 7   Per Capita Consum  25569 non-null  float64
 8   P_origen           25569 non-null  object 
 9   Value              25569 non-null  object 
 10  Cases              25569 non-null  float64
 11  Deaths             25569 non-null  float64
 12  Cases + Deaths     25569 non-null  float64
dtypes: float64(8), int64(2), object(3)
memory usage: 2.7+ MB


# Export

In [ ]:
df.to_csv("dataset_final.csv", index=False)